In [1]:
# 1 导入包
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import normalize
from keras.layers import LSTM
from keras.layers import Embedding, SimpleRNN

In [2]:
# 2 读取数据集

# dataframe = pd.read_csv("total.csv",header=None).values # 取消第一行作为表头
train_set = pd.read_csv("train_set.csv")
test_set = pd.read_csv("test_set.csv")
print(train_set)

# 3 读标签
# label = dataframe.loc[:,['label']]
y_train = train_set['label']
test_label = test_set['label']

print(y_train)
print("y_train.shape",y_train)

        label  frame.len  ip.hdr_len  ip.dsfield  ip.len  ip.flags.rb  \
0           0         66          20           0      52            0   
1           0         66          20           0      52            0   
2           0         66          20           0      52            0   
3           0         66          20           0      52            0   
4           0         66          20           0      52            0   
...       ...        ...         ...         ...     ...          ...   
837180     22         74          20           0      60            0   
837181     22         74          20           0      60            0   
837182     22         74          20          40      60            0   
837183     22         66          20           0      52            0   
837184     22         74          20           0      60            0   

        ip.flags.df  ip.flags.mf  ip.frag_offset  ip.ttl  ...  \
0                 0            0               0      63  

In [3]:

# 4 筛选GA特征
x_train_set_GA = train_set[['frame.len', 'ip.len', 'ip.flags.df', 'ip.flags.mf', 'ip.ttl',
       'ip.proto', 'tcp.hdr_len', 'tcp.flags.ns', 'tcp.flags.ecn',
       'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset',
       'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size_value',
       'tcp.urgent_pointer', 'tcp.options.wscale.shift', 'tcp.options.mss_val',
       'tcp.options.timestamp.tsval', 'tcp.options.timestamp.tsecr',
       'udp.length', 'udp.checksum', 'tcp.option_kind',
       '_ws.col.Protocol_AJP13', '_ws.col.Protocol_BGP',
       '_ws.col.Protocol_BZR', '_ws.col.Protocol_DRDA', '_ws.col.Protocol_DSI',
       '_ws.col.Protocol_FTP', '_ws.col.Protocol_GIOP',
       '_ws.col.Protocol_GTPv2', '_ws.col.Protocol_Gopher',
       '_ws.col.Protocol_Gryphon', '_ws.col.Protocol_HTTP',
       '_ws.col.Protocol_HTTP/JSON', '_ws.col.Protocol_HTTP/XML',
       '_ws.col.Protocol_ICAP', '_ws.col.Protocol_ICMP',
       '_ws.col.Protocol_Kafka', '_ws.col.Protocol_LDAP',
       '_ws.col.Protocol_MySQL', '_ws.col.Protocol_NBNS',
       '_ws.col.Protocol_NDMP', '_ws.col.Protocol_NNTP',
       '_ws.col.Protocol_POP', '_ws.col.Protocol_Portmap',
       '_ws.col.Protocol_R3', '_ws.col.Protocol_RPC', '_ws.col.Protocol_RSYNC',
       '_ws.col.Protocol_RTMP', '_ws.col.Protocol_RTSP',
       '_ws.col.Protocol_SABP', '_ws.col.Protocol_SMTP',
       '_ws.col.Protocol_SSH', '_ws.col.Protocol_SSLv2',
       '_ws.col.Protocol_SSLv3', '_ws.col.Protocol_TCP',
       '_ws.col.Protocol_TCPCL', '_ws.col.Protocol_TDS',
       '_ws.col.Protocol_TELNET', '_ws.col.Protocol_TLSv1',
       '_ws.col.Protocol_TLSv1.1', '_ws.col.Protocol_TLSv1.2',
       '_ws.col.Protocol_TN3270', '_ws.col.Protocol_TPKT',
       '_ws.col.Protocol_UDP', '_ws.col.Protocol_VICP',
       '_ws.col.Protocol_ZEBRA']]
test_set = test_set[['frame.len', 'ip.len', 'ip.flags.df', 'ip.flags.mf', 'ip.ttl',
       'ip.proto', 'tcp.hdr_len', 'tcp.flags.ns', 'tcp.flags.ecn',
       'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset',
       'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size_value',
       'tcp.urgent_pointer', 'tcp.options.wscale.shift', 'tcp.options.mss_val',
       'tcp.options.timestamp.tsval', 'tcp.options.timestamp.tsecr',
       'udp.length', 'udp.checksum', 'tcp.option_kind',
       '_ws.col.Protocol_AJP13', '_ws.col.Protocol_BGP',
       '_ws.col.Protocol_BZR', '_ws.col.Protocol_DRDA', '_ws.col.Protocol_DSI',
       '_ws.col.Protocol_FTP', '_ws.col.Protocol_GIOP',
       '_ws.col.Protocol_GTPv2', '_ws.col.Protocol_Gopher',
       '_ws.col.Protocol_Gryphon', '_ws.col.Protocol_HTTP',
       '_ws.col.Protocol_HTTP/JSON', '_ws.col.Protocol_HTTP/XML',
       '_ws.col.Protocol_ICAP', '_ws.col.Protocol_ICMP',
       '_ws.col.Protocol_Kafka', '_ws.col.Protocol_LDAP',
       '_ws.col.Protocol_MySQL', '_ws.col.Protocol_NBNS',
       '_ws.col.Protocol_NDMP', '_ws.col.Protocol_NNTP',
       '_ws.col.Protocol_POP', '_ws.col.Protocol_Portmap',
       '_ws.col.Protocol_R3', '_ws.col.Protocol_RPC', '_ws.col.Protocol_RSYNC',
       '_ws.col.Protocol_RTMP', '_ws.col.Protocol_RTSP',
       '_ws.col.Protocol_SABP', '_ws.col.Protocol_SMTP',
       '_ws.col.Protocol_SSH', '_ws.col.Protocol_SSLv2',
       '_ws.col.Protocol_SSLv3', '_ws.col.Protocol_TCP',
       '_ws.col.Protocol_TCPCL', '_ws.col.Protocol_TDS',
       '_ws.col.Protocol_TELNET', '_ws.col.Protocol_TLSv1',
       '_ws.col.Protocol_TLSv1.1', '_ws.col.Protocol_TLSv1.2',
       '_ws.col.Protocol_TN3270', '_ws.col.Protocol_TPKT',
       '_ws.col.Protocol_UDP', '_ws.col.Protocol_VICP',
       '_ws.col.Protocol_ZEBRA']]

print(x_train_set_GA)
print("筛选GA特征",x_train_set_GA.shape) # 筛选了69个特征

########## 一般是转为浮点数向量

# train_set = np.array(train_set[1:,:]).astype(np.float32) # 去掉第一行字段 
# test_set = np.array(test_set[1:,:]).astype(np.float32)

train_set_GA = np.array(x_train_set_GA).astype(np.float32) 
test_set = np.array(test_set).astype(np.float32)

########



        frame.len  ip.len  ip.flags.df  ip.flags.mf  ip.ttl  ip.proto  \
0              66      52            0            0      63         6   
1              66      52            0            0      63         6   
2              66      52            0            0      63         6   
3              66      52            0            0      63         6   
4              66      52            0            0      63         6   
...           ...     ...          ...          ...     ...       ...   
837180         74      60            0            0      56         6   
837181         74      60            0            0      49         6   
837182         74      60            1            0      33         6   
837183         66      52            1            0     128         6   
837184         74      60            1            0      52         6   

        tcp.hdr_len  tcp.flags.ns  tcp.flags.ecn  tcp.flags.urg  ...  \
0              32.0           0.0            0.0   

In [4]:
# 5 特征归一化  用MLP的时候用，LSTM不用
train_set_GA = normalize(train_set_GA, axis=0, norm='max')
x_test = normalize(test_set, axis=0, norm='max')

print("x_train_GA.shape",train_set_GA.shape)


# 6 标签转化为one-hot编码 
y_train = keras.utils.to_categorical(y_train, num_classes=23)
y_test = keras.utils.to_categorical(test_label, num_classes=23)

print("y_train.shape",y_train.shape)

x_train_GA.shape (837185, 69)
y_train.shape (837185, 23)


In [6]:
#################  多层感知机

model = Sequential()
# Dense(64) 是一个具有 64 个隐藏神经元的全连接层。
# 在第一层必须指定所期望的输入数据尺寸：
# 在这里，是一个104 维的向量。
model.add(Dense(64, activation='relu', input_dim=69))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(23, activation='softmax'))


In [7]:

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_set_GA, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

##################

Epoch 1/20
6541/6541 [==============================] - 12s 2ms/step - loss: 1.8243 - accuracy: 0.3179
Epoch 2/20
6541/6541 [==============================] - 12s 2ms/step - loss: 1.7265 - accuracy: 0.3437
Epoch 3/20
6541/6541 [==============================] - 12s 2ms/step - loss: 1.7208 - accuracy: 0.3477
Epoch 4/20
6541/6541 [==============================] - 11s 2ms/step - loss: 1.7231 - accuracy: 0.3487
Epoch 5/20
6541/6541 [==============================] - 11s 2ms/step - loss: 1.7305 - accuracy: 0.3482
Epoch 6/20
6541/6541 [==============================] - 11s 2ms/step - loss: 1.7395 - accuracy: 0.3465
Epoch 7/20
6541/6541 [==============================] - 11s 2ms/step - loss: 1.7525 - accuracy: 0.3446
Epoch 8/20
6541/6541 [==============================] - 11s 2ms/step - loss: 1.7590 - accuracy: 0.3425
Epoch 9/20
6541/6541 [==============================] - 11s 2ms/step - loss: 1.7703 - accuracy: 0.3394
Epoch 10/20
6541/6541 [==============================] - 12s 2ms/step - l